In [1]:
# Check some of the microflare results
# 
# 05-Dec-2023 IGH

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
from sunpy.io.special import read_genx
from astropy.time import Time
import scipy.special as ss

import warnings
warnings.simplefilter('ignore')

matplotlib.rcParams['font.size']=16

In [3]:
# From sswidl this file was made as:
# rr={fstart:anytim(res.fstart,/ccsds),fend:anytim(res.fend,/ccsds),fpeak:anytim(res.fpeak,/ccsds),$
#     fpeak_tr:anytim(res.fpeak_tr,/ccsds),bk_bf_tr:anytim(res.bk_bf_tr,/ccsds),$
#     tmk:res.osx_p[1],em:res.osx_p[0]*1d49,$
#     norm:res.osx_p[3],g1:1.5,eb:res.osx_p[5],g2:res.osx_p[6],$
#     ec:ec,ph4_8:ph4_8,ph12:ph12,vol:res.vf_vol,vflx4_8:res.vf_fit.srcflux,vx:res.vf_fit.srcx,vy:res.vf_fit.srcy,$
#     gflx_bs:res.gflx_bs,gtmk:tems.tmk,gem:tems.em,$
#     eng_th:res.eng_th,eng_nn:res.eng_nn,$
#     idgdth:gdth,idgdnn:gdnn,idgsgdth:gsgdth,idgsgdnn:gsgdnn}
# 
# Note that osx_p[1] was already converted from tkev to TMK via osx_p[1]/0.086164

res=read_genx('../wee_2008/wee_all.genx')
pktim=res["SAVEGEN0"]["FPEAK"]

In [4]:
apktim=Time(pktim,format='isot')

In [5]:
wtims=["2003-02-27 06:22:34.000",
       "2003-03-17 18:41:46.000",
       "2003-04-29 17:43:02.000",
       "2003-07-25 08:26:42.000",
       "2004-01-17 07:28:46.000",
       "2004-10-24 00:31:46.000"]
nf=len(wtims)

In [6]:
# Find the microflares in the main list
ids=np.ones(nf,dtype=int)

for i in range(nf):
    temp=Time(wtims[i],format='iso')
    ids[i]=np.where(apktim.decimalyear == temp.decimalyear)[0]

print(ids)

[ 6507  7066  8150 10032 13408 17602]


In [7]:
# Check the results

for i in range(nf):
    print(wtims[i]+"  ---   "+pktim[ids[i]])

2003-02-27 06:22:34.000  ---   2003-02-27T06:22:34.000
2003-03-17 18:41:46.000  ---   2003-03-17T18:41:46.000
2003-04-29 17:43:02.000  ---   2003-04-29T17:43:02.000
2003-07-25 08:26:42.000  ---   2003-07-25T08:26:42.000
2004-01-17 07:28:46.000  ---   2004-01-17T07:28:46.000
2004-10-24 00:31:46.000  ---   2004-10-24T00:31:46.000


In [8]:
# Do these have good ids?
# Events with "good" fits (th, nn, th+gs, nn+gs)
idgdth=res["SAVEGEN0"]["IDGDTH"]
idgdnn=res["SAVEGEN0"]["IDGDNN"]
idgsgdth=res["SAVEGEN0"]["IDGSGDTH"]
idgsgdnn=res["SAVEGEN0"]["IDGSGDNN"]


gdth=np.isin(ids,idgdth)
gdnn=np.isin(ids,idgdnn)
gdgsth=np.isin(ids,idgsgdth)
gdgsnn=np.isin(ids,idgsgdnn)

for i in range(nf):
    print(wtims[i])
    print(f"Gd th    fit: {gdth[i]}")
    print(f"Gd th+nn fit: {gdnn[i]}")
    print(f"Gd GOES & th    fit: {gdgsth[i]}")
    print(f"Gd GOES & th+nn fit: {gdgsnn[i]}")

# OK so one with bad fit, 29-Apr-2003 is not flagged as good
# 24-Oct-2004 only good RHESSI fit but bad GOES data (which is fine)


2003-02-27 06:22:34.000
Gd th    fit: True
Gd th+nn fit: True
Gd GOES & th    fit: True
Gd GOES & th+nn fit: True
2003-03-17 18:41:46.000
Gd th    fit: True
Gd th+nn fit: True
Gd GOES & th    fit: True
Gd GOES & th+nn fit: True
2003-04-29 17:43:02.000
Gd th    fit: False
Gd th+nn fit: False
Gd GOES & th    fit: False
Gd GOES & th+nn fit: False
2003-07-25 08:26:42.000
Gd th    fit: True
Gd th+nn fit: True
Gd GOES & th    fit: True
Gd GOES & th+nn fit: True
2004-01-17 07:28:46.000
Gd th    fit: True
Gd th+nn fit: True
Gd GOES & th    fit: True
Gd GOES & th+nn fit: True
2004-10-24 00:31:46.000
Gd th    fit: True
Gd th+nn fit: True
Gd GOES & th    fit: False
Gd GOES & th+nn fit: False


In [9]:
# What were the fit results
# OSPEX fit of f_vth + bpow
tmk=res["SAVEGEN0"]["TMK"]
em=res["SAVEGEN0"]["EM"]
norm=res["SAVEGEN0"]["NORM"]
eb=res["SAVEGEN0"]["EB"]
g2=res["SAVEGEN0"]["G2"]
# Estimated low energy cutoff from bpow fit
ec=res["SAVEGEN0"]["EC"]
# The pre-calculated energies
ength=res["SAVEGEN0"]["ENG_TH"]
engnn=res["SAVEGEN0"]["ENG_NN"]
vol=res["SAVEGEN0"]["VOL"]

In [10]:
for i in range(nf):
    print('-----')
    print(wtims[i])
    print(f'T: {tmk[ids[i]]:.2f} MK, EM {em[ids[i]]:.2e}')
    print(f'EB {eb[ids[i]]:.2f} keV, gam {g2[ids[i]]:.2f}')
    print(f'EC: {ec[ids[i]]:.2f} keV, Norm: {norm[ids[i]]:.2f}')

-----
2003-02-27 06:22:34.000
T: 12.43 MK, EM 1.42e+46
EB 9.00 keV, gam 5.51
EC: 12.27 keV, Norm: 0.05
-----
2003-03-17 18:41:46.000
T: 11.95 MK, EM 2.48e+46
EB 8.28 keV, gam 5.42
EC: 11.12 keV, Norm: 0.35
-----
2003-04-29 17:43:02.000
T: 14.12 MK, EM 2.19e+46
EB 9.03 keV, gam 6.98
EC: 12.01 keV, Norm: 0.33
-----
2003-07-25 08:26:42.000
T: 12.96 MK, EM 3.12e+46
EB 8.55 keV, gam 5.75
EC: 11.49 keV, Norm: 0.52
-----
2004-01-17 07:28:46.000
T: 13.69 MK, EM 1.74e+46
EB 9.42 keV, gam 6.40
EC: 12.77 keV, Norm: 0.15
-----
2004-10-24 00:31:46.000
T: 13.75 MK, EM 1.05e+46
EB 7.96 keV, gam 5.11
EC: 10.63 keV, Norm: 0.49


In [11]:
for i in range(nf):
    print('-----')
    print(wtims[i])
    j=ids[i]
    print(f'Vol: {vol[j]:.2e} cm^3')
    tempth=3*np.sqrt(em[j]*vol[j])*1.3806503e-23*tmk[j]*1e6*1e7 #convert MK to K, and J to erg
    print(f'File ength: {ength[j]:.2e} erg, Calc ength: {tempth:.2e} erg')
    # Norm value is below the break power-law extended to 50 keV, so convert back to Eb value, i.e.
    # https://soho.nascom.nasa.gov/solarsoft/packages/xray/idl/f_bpow.pro
    feb=norm[j]*(eb[j]/50)**-1.5 
    # Then work out power-law above the break down to 1 keV, as needed for calc
    f1=feb*eb[j]**g2[j]
    # 16* as power over 16s integration
    tempnn=16*9.5e24*g2[j]**2*(g2[j]-1)*ss.beta(g2[j]-0.5,1.5)*f1*eb[j]**(1-g2[j])
    print(f'File engnn: {engnn[j]:.2e} erg, Calc tempnn: {tempnn:.2e} erg')

# So inconsistent values in the combined new file so need to check what comes from where, 
# as given energies don't match ospex parameters and vol for one is 0, hence the thermal energy is 0! 


-----
2003-02-27 06:22:34.000
Vol: 9.77e+25 cm^3
File ength: 4.40e+27 erg, Calc ength: 6.07e+27 erg
File engnn: 9.43e+27 erg, Calc tempnn: 8.35e+27 erg
-----
2003-03-17 18:41:46.000
Vol: 1.57e+26 cm^3
File ength: 1.65e+28 erg, Calc ength: 9.77e+27 erg
File engnn: 6.67e+28 erg, Calc tempnn: 6.40e+28 erg
-----
2003-04-29 17:43:02.000
Vol: 7.43e+26 cm^3
File ength: 1.04e+28 erg, Calc ength: 2.36e+28 erg
File engnn: 9.53e+28 erg, Calc tempnn: 8.70e+28 erg
-----
2003-07-25 08:26:42.000
Vol: 4.25e+26 cm^3
File ength: 1.03e+28 erg, Calc ength: 1.96e+28 erg
File engnn: 9.67e+28 erg, Calc tempnn: 1.04e+29 erg
-----
2004-01-17 07:28:46.000
Vol: 2.59e+26 cm^3
File ength: 6.72e+27 erg, Calc ength: 1.20e+28 erg
File engnn: 3.73e+28 erg, Calc tempnn: 3.42e+28 erg
-----
2004-10-24 00:31:46.000
Vol: 0.00e+00 cm^3
File ength: 3.03e+27 erg, Calc ength: 0.00e+00 erg
File engnn: 7.61e+28 erg, Calc tempnn: 8.24e+28 erg


In [15]:
# What about the paper values from Fig 9 ??
ptmk=np.array([12.21,11.62,13.75,12.76,13.38,13.47])
pem=np.array([1.57e46,2.94e46,2.5e46,3.46e46,1.96e46,1.17e46])

pfeb=np.array([.69,5.32,4.64,6.4,2.01,7.23])
pg2=np.array([5.54,5.46,7.05,5.88,6.45,5.15])
peb=np.array([8.83,8.28,8.98,8.85,9.36,8.08])

for i in range(nf):
    print('-----')
    print(wtims[i])
    j=ids[i]
    print(f'Vol: {vol[j]:.2e} cm^3')
    tempth=3*np.sqrt(pem[i]*vol[j])*1.3806503e-23*ptmk[i]*1e6*1e7 #convert MK to K, and J to erg
    print(f'File ength: {ength[j]:.2e} erg, Paper Calc ength: {tempth:.2e} erg')

    f1=pfeb[i]*peb[i]**pg2[i]
    # 16* as power over 16s integration
    tempnn=16*9.5e24*pg2[i]**2*(pg2[i]-1)*ss.beta(pg2[i]-0.5,1.5)*f1*peb[i]**(1-pg2[i])
    print(f'File engnn: {engnn[j]:.2e} erg, Paper Calc tempnn: {tempnn:.2e} erg')

-----
2003-02-27 06:22:34.000
Vol: 9.77e+25 cm^3
File ength: 4.40e+27 erg, Paper Calc ength: 6.26e+27 erg
File engnn: 9.43e+27 erg, Paper Calc tempnn: 9.43e+27 erg
-----
2003-03-17 18:41:46.000
Vol: 1.57e+26 cm^3
File ength: 1.65e+28 erg, Paper Calc ength: 1.03e+28 erg
File engnn: 6.67e+28 erg, Paper Calc tempnn: 6.65e+28 erg
-----
2003-04-29 17:43:02.000
Vol: 7.43e+26 cm^3
File ength: 1.04e+28 erg, Paper Calc ength: 2.45e+28 erg
File engnn: 9.53e+28 erg, Paper Calc tempnn: 9.53e+28 erg
-----
2003-07-25 08:26:42.000
Vol: 4.25e+26 cm^3
File ength: 1.03e+28 erg, Paper Calc ength: 2.03e+28 erg
File engnn: 9.67e+28 erg, Paper Calc tempnn: 9.66e+28 erg
-----
2004-01-17 07:28:46.000
Vol: 2.59e+26 cm^3
File ength: 6.72e+27 erg, Paper Calc ength: 1.25e+28 erg
File engnn: 3.73e+28 erg, Paper Calc tempnn: 3.73e+28 erg
-----
2004-10-24 00:31:46.000
Vol: 0.00e+00 cm^3
File ength: 3.03e+27 erg, Paper Calc ength: 0.00e+00 erg
File engnn: 7.61e+28 erg, Paper Calc tempnn: 8.01e+28 erg
